In [1]:
!pip install transformers
!pip install sentencepiece
!git clone https://github.com/amontgomerie/question_generator
!pip install rouge
!pip install sentence-transformers

     |████████████████████████████████| 2.6 MB 33.8 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 636 kB 49.3 MB/s 
     |████████████████████████████████| 3.3 MB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 16.9 MB/s 
Cloning into 'question_generator'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 199 (delta 45), reused 24 (delta 9), pack-reused 112
Receiving objects: 100% (199/199), 101.67 KiB | 16.95 MiB/s, done.
Resolving deltas: 100% (100/100), done.
     |████████████████████████████████| 85 kB 4.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=af31838f84972d9a6ba6c1f68fccd10af0086dd2834

In [2]:
import pandas as pd
from question_generator.questiongenerator import QuestionGenerator
from nltk.translate.bleu_score import sentence_bleu # BLEU score
from rouge import Rouge # Rouge score
from sentence_transformers import SentenceTransformer, util # Sentence-BERT

In [3]:
# Question Generator prepare
qg = QuestionGenerator()
# evaluation prepare
rouge = Rouge()
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [4]:
# 讀入SQuAD問答資料集
df = pd.read_excel("SQuAD.xlsx")

# 只擷取SQuAD問答資料集的部分資料做測試
df_part = df[1:754]
data = df_part.groupby(['Column2'])
output_data = {"context":[],
              "SQuAD_Question":[],
              "score_bleu":[],
              "generated question_bleu":[],
              "score_Sentence-BERT":[],
              "generated question_Sentence-BERT":[],
              "score_rouge-1":[],        
              "generated question_rouge-1":[],
              "score_rouge-l":[],              
              "generated question_rouge-l":[],
              }

In [5]:
for d in data: # for "every paragraph"
    paragraph = d[0]
    squadquestion = d[1]['Column3']
    # print('paragraph\n', paragraph)
    allqglist = []
    qg_ans = qg.generate(paragraph, num_questions=6, answer_style='sentences')
    # generated question preprocessing
    for q in qg_ans:
        allqglist.append(q['question'])
    ## for "every SQuAD question" according to certain paragraph
    for stdquestion in squadquestion:
        #reference = stdquestion[0:-1].split() # use [0:-1] is to eliminate '?'
        reference = stdquestion[0:-1].split('?')[0]
        reference_split = reference.split()
        # Sentences are encoded by calling model.encode() # Sentence-BERT
        emb1 = model.encode(reference)
        ## data for recording
        counter = 0
        # part of bleu
        maxscore_bleu = 0
        maxidx_bleu = 0
        # part of Sentence-BERT
        maxscore_Sentence_BERT = 0
        maxidx_Sentence_BERT = 0
        # part of rouge-1
        maxscore_rouge_1 = 0
        maxidx_rouge_1 = 0
        # part of rouge-l
        maxscore_rouge_l = 0
        maxidx_rouge_l = 0
        ## for "every generated question" according to certain paragraph
        for qgquestion in allqglist:
            candidate = qgquestion[0:-1].split('?')[0] # use [0:-1] is to eliminate '?'
            candidate_split = candidate.split()
            
            emb2 = model.encode(candidate)
            # caculate BLEU score
            scores_bleu = sentence_bleu([reference_split], candidate_split, weights=(1.0, 0, 0, 0))
            # caculate Sentence-BERT score
            cos_sim = util.pytorch_cos_sim(emb1, emb2).tolist()[0][0]
            # caculate rouge score
            scores_rouge = rouge.get_scores(reference, candidate)
            
            ## record the highest score of each evaluation
            # part of BLEU score record
            if scores_bleu > maxscore_bleu: # record highest score
                maxscore_bleu = scores_bleu
                maxidx_bleu = counter
            # part of Sentence-BERT score record
            if cos_sim > maxscore_Sentence_BERT: # record highest score
                maxscore_Sentence_BERT = cos_sim
                maxidx_Sentence_BERT = counter            
            # part of rouge-1 record
            if scores_rouge[0]['rouge-1']['r'] > maxscore_rouge_1: # record highest score
                maxscore_rouge_1 = scores_rouge[0]['rouge-1']['r']
                maxidx_rouge_1 = counter
            # part of rouge-l record
            if scores_rouge[0]['rouge-l']['r'] > maxscore_rouge_l: # record highest score
                maxscore_rouge_l = scores_rouge[0]['rouge-l']['r']
                maxidx_rouge_l = counter            
            counter = counter+1
            
        ## record the sentense and score in data frame (for excel output)
        output_data["context"].append(paragraph)
        output_data["SQuAD_Question"].append(stdquestion)
        # part of BLEU   
        output_data["score_bleu"].append(maxscore_bleu)        
        output_data["generated question_bleu"].append(allqglist[maxidx_bleu])
        # part of Sentence-BERT               
        output_data["score_Sentence-BERT"].append(maxscore_Sentence_BERT)        
        output_data["generated question_Sentence-BERT"].append(allqglist[maxidx_Sentence_BERT])
        # part of rouge-1   
        output_data["score_rouge-1"].append(maxscore_rouge_1)        
        output_data["generated question_rouge-1"].append(allqglist[maxidx_rouge_1])
        # part of rouge-l   
        output_data["score_rouge-l"].append(maxscore_rouge_l)        
        output_data["generated question_rouge-l"].append(allqglist[maxidx_rouge_l])
        # testing
        # print('scores_bleu:', maxscore_bleu, 'scores_Sentence-BERT:', maxscore_Sentence_BERT, 'score_rouge1:', maxscore_rouge_1, 'score_rougel:', maxscore_rouge_l )
    
    print('Finished entry:', len(output_data["SQuAD_Question"]), ' Progressing rate:', len(output_data["SQuAD_Question"])/753)

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Finished entry: 12  Progressing rate: 0.01593625498007968
Generating questions...

Evaluating QA pairs...


Was only able to generate 4 questions. For more questions, please input a longer text.
Finished entry: 19  Progressing rate: 0.025232403718459494
Generating questions...

Evaluating QA pairs...


Was only able to generate 2 questions. For more questions, please input a longer text.
Finished entry: 28  Progressing rate: 0.03718459495351926
Generating questions...

Evaluating QA pairs...

Finished entry: 41  Progressing rate: 0.05444887118193891
Generating questions...

Evaluating QA pairs...

Finished entry: 55  Progressing rate: 0.07304116865869854
Generating questions...

Evaluating QA pairs...

Finished entry: 71  Progressing rate: 0.09428950863213811
Generating questions...

Evaluating QA pairs...

Finished entry: 84  Progressing rate: 0.11155378486055777
Generating questions...

Evaluating QA pairs...


Was only able to generate 4 questions. For more questions, please input a

In [6]:
## output           
excel_output = pd.DataFrame(output_data)
excel_output.to_excel('qg_score.xlsx')